# Misurare l'associazione fra variabili categoriche

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats.contingency import margins

# Carico i dati dal file Excel, specificando quale worksheet
df = pd.read_excel('data/chisquare_preferenza_genere_prodotto.xlsx', sheet_name="Data")


## Tabella di contingenza

In [ ]:
# Tabelle di contingenza:
contingency_table = pd.crosstab(df['gender'], df['productid'], margins=True, margins_name="Totale")
print(contingency_table)

## Probabilità Marginale
È la probabilità di un singolo evento senza considerare l'effetto di un altro evento. Si tratta essenzialmente della probabilità "a margine" di un evento particolare, quindi senza condizionarlo su un altro. 
Ad esempio, considera una tabella di contingenza tra Genere (Maschio/Femmina) e Prodotto (1/2/3). La probabilità marginale che una persona scelta a caso sia un maschio (indipendentemente dal fatto che fumi o meno) si calcola sommando tutte le osservazioni dei maschi e dividendo per il totale delle osservazioni.

In [ ]:
# Probabilità marginale:
prob_gender = df['gender'].value_counts() / len(df)
prob_productid = df['productid'].value_counts() / len(df)
print("\nProbabilità marginale di genere:\n", prob_gender)
print("\nProbabilità marginale di prodotto scelto:\n", prob_productid)

# Test del chi-quadrato

In [ ]:
# Test chi square: utilizzo i -1 per non prendere l'ultima riga e l'ultima colonna
chi2, p, _, expected = chi2_contingency(contingency_table.iloc[:-1, :-1])

print("\nValore chi2:", chi2)
print("P-value:", p)
print("\nTabella delle frequenze attese:\n", pd.DataFrame(expected, columns=contingency_table.columns[:-1], index=contingency_table.index[:-1]))


## Residual analysis

In [ ]:
def stdres(observed, expected):
    n = observed.sum()
    rsum, csum = margins(observed)
    # With integers, the calculation
    #     csum * rsum * (n - rsum) * (n - csum)
    # might overflow, so convert rsum and csum to floating point.
    rsum = rsum.astype(np.float64)
    csum = csum.astype(np.float64)
    v = csum * rsum * (n - rsum) * (n - csum) / n**3
    return (observed - expected) / np.sqrt(v)

residuals = stdres(contingency_table.iloc[:-1, :-1].values, expected)
print("\nStandardized Residuals:\n", residuals)